In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('isp20-21data.txt')
winter2 = pd.read_csv ('isp21-22data.txt')
winter3 = pd.read_csv ('isp22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
205,ISP,2021-12-08 15:56:00,0.51,-RASN
206,ISP,2021-12-08 16:56:00,1.27,-RASN BR
207,ISP,2021-12-08 17:53:00,1.02,-RASN BR
208,ISP,2021-12-08 17:56:00,1.02,-RASN BR
219,ISP,2021-12-09 01:36:00,T,-SN BR
...,...,...,...,...
2306,ISP,2022-02-19 21:14:00,T,-SN
2307,ISP,2022-02-19 21:18:00,T,-SN
2308,ISP,2022-02-19 21:28:00,T,SN FG
2309,ISP,2022-02-19 21:37:00,T,-SN FZFG


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
isp = LocationInfo(
    'Islip' 'New York', 
    latitude= 40.7298,
    longitude= -73.2106, 
    timezone='America/New_York'
)

isp

LocationInfo(name='IslipNew York', region='England', timezone='America/New_York', latitude=40.7298, longitude=-73.2106)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(isp.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(isp.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(isp.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:58:20  2020-12-01 16:25:40  2020-12-01
1   2020-12-02 06:59:20  2020-12-02 16:25:27  2020-12-02
2   2020-12-03 07:00:19  2020-12-03 16:25:16  2020-12-03
3   2020-12-04 07:01:17  2020-12-04 16:25:08  2020-12-04
4   2020-12-05 07:02:14  2020-12-05 16:25:02  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:34:06  2021-02-24 17:38:26  2021-02-24
86  2021-02-25 06:32:37  2021-02-25 17:39:35  2021-02-25
87  2021-02-26 06:31:08  2021-02-26 17:40:45  2021-02-26
88  2021-02-27 06:29:38  2021-02-27 17:41:54  2021-02-27
89  2021-02-28 06:28:07  2021-02-28 17:43:02  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:58:05  2021-12-01 16:25:43  2021-12-01
1   2021-12-02 06:59:06  2021-12-02 16:25:30  2021-12-02
2   2021-12-03 07:00:05  2021-12-03 16:25:18  2021-12-03
3   2021-12-04 07:01:03  2021-12-04 16:25:09  2021-12-04
4   2021-12-05 07:02:01  2021-12-05 16:25:03  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:34:27  2022-02-24 17:38:09  2022-02-24
86  2022-02-25 06:32:59  2022-02-25 17:39:19  2022-02-25
87  2022-02-26 06:31:30  2022-02-26 17:40:28  2022-02-26
88  2022-02-27 06:30:00  2022-02-27 17:41:37  2022-02-27
89  2022-02-28 06:28:29  2022-02-28 17:42:46  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:57:50  2022-12-01 16:25:46  2022-12-01
1   2022-12-02 06:58:51  2022-12-02 16:25:32  2022-12-02
2   2022-12-03 06:59:51  2022-12-03 16:25:21  2022-12-03
3   2022-12-04 07:00:49  2022-12-04 16:25:11  2022-12-04
4   2022-12-05 07:01:47  2022-12-05 16:25:04  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:34:48  2023-02-24 17:37:52  2023-02-24
86  2023-02-25 06:33:20  2023-02-25 17:39:02  2023-02-25
87  2023-02-26 06:31:51  2023-02-26 17:40:11  2023-02-26
88  2023-02-27 06:30:22  2023-02-27 17:41:20  2023-02-27
89  2023-02-28 06:28:51  2023-02-28 17:42:29  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ISP,2020-12-09 19:56:00,0.0001,-SN,2020-12-09,2020-12-09 07:05:49,2020-12-09 16:25:00
1,ISP,2020-12-09 20:56:00,0.0001,-SN,2020-12-09,2020-12-09 07:05:49,2020-12-09 16:25:00
2,ISP,2020-12-16 20:56:00,0.2500,-SN,2020-12-16,2020-12-16 07:11:05,2020-12-16 16:26:26
3,ISP,2020-12-16 21:56:00,1.0200,-SN BR,2020-12-16,2020-12-16 07:11:05,2020-12-16 16:26:26
4,ISP,2020-12-16 22:56:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:11:05,2020-12-16 16:26:26
...,...,...,...,...,...,...,...
117,ISP,2021-02-20 01:56:00,0.2500,-SN BR,2021-02-20,2021-02-20 06:39:51,2021-02-20 17:33:45
118,ISP,2021-02-20 11:56:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:39:51,2021-02-20 17:33:45
119,ISP,2021-02-20 12:56:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:39:51,2021-02-20 17:33:45
120,ISP,2021-02-20 13:56:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:39:51,2021-02-20 17:33:45


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ISP,2021-12-08 15:56:00,0.5100,-RASN,2021-12-08,2021-12-08 07:04:44,2021-12-08 16:24:57
1,ISP,2021-12-08 16:56:00,1.2700,-RASN BR,2021-12-08,2021-12-08 07:04:44,2021-12-08 16:24:57
2,ISP,2021-12-08 17:56:00,1.0200,-RASN BR,2021-12-08,2021-12-08 07:04:44,2021-12-08 16:24:57
3,ISP,2021-12-09 02:56:00,0.5100,-SN BR,2021-12-09,2021-12-09 07:05:36,2021-12-09 16:24:59
4,ISP,2021-12-24 08:56:00,0.2500,-SN BR,2021-12-24,2021-12-24 07:15:11,2021-12-24 16:30:05
...,...,...,...,...,...,...,...
92,ISP,2022-02-13 23:56:00,0.0001,-SN BR,2022-02-13,2022-02-13 06:49:33,2022-02-13 17:25:04
93,ISP,2022-02-14 00:56:00,0.5100,-SN BR,2022-02-14,2022-02-14 06:48:16,2022-02-14 17:26:17
94,ISP,2022-02-14 01:56:00,0.2500,-SN BR,2022-02-14,2022-02-14 06:48:16,2022-02-14 17:26:17
95,ISP,2022-02-14 02:56:00,0.0001,-SN,2022-02-14,2022-02-14 06:48:16,2022-02-14 17:26:17


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ISP,2022-12-12 02:56:00,0.0001,-SN,2022-12-12,2022-12-12 07:07:51,2022-12-12 16:25:18
1,ISP,2022-12-12 04:56:00,1.0200,-SNRA BR,2022-12-12,2022-12-12 07:07:51,2022-12-12 16:25:18
2,ISP,2022-12-12 12:56:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:07:51,2022-12-12 16:25:18
3,ISP,2022-12-12 13:56:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:07:51,2022-12-12 16:25:18
4,ISP,2023-01-15 04:56:00,0.0001,-SN,2023-01-15,2023-01-15 07:15:08,2023-01-15 16:49:38
5,ISP,2023-01-23 19:56:00,0.7600,-SN BR,2023-01-23,2023-01-23 07:10:45,2023-01-23 16:58:58
6,ISP,2023-01-23 21:56:00,0.5100,-RASN BR,2023-01-23,2023-01-23 07:10:45,2023-01-23 16:58:58
7,ISP,2023-01-23 22:56:00,0.7600,-RASN BR,2023-01-23,2023-01-23 07:10:45,2023-01-23 16:58:58
8,ISP,2023-01-27 18:56:00,0.2500,-SNRA,2023-01-27,2023-01-27 07:07:44,2023-01-27 17:03:50
9,ISP,2023-01-31 12:56:00,0.2500,-RAPL,2023-01-31,2023-01-31 07:04:14,2023-01-31 17:08:45


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
11,ISP,2020-12-17 10:56:00,3.3000,SNPL FZFG BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47
12,ISP,2020-12-17 11:56:00,1.0200,+SN FZFG BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47
13,ISP,2020-12-17 12:56:00,0.5100,-SN FZFG BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47
14,ISP,2020-12-17 13:56:00,0.2500,-SN FZFG BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47
15,ISP,2020-12-17 14:56:00,0.0001,-SN BR BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47
16,ISP,2020-12-17 15:56:00,0.0001,-SN BR BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47
22,ISP,2021-01-03 15:56:00,0.2500,-PLRA,2021-01-03,2021-01-03 07:17:22,2021-01-03 16:37:52
23,ISP,2021-01-20 12:56:00,0.0001,-SN,2021-01-20,2021-01-20 07:12:22,2021-01-20 16:55:58
24,ISP,2021-01-20 13:56:00,0.5100,-SN BR,2021-01-20,2021-01-20 07:12:22,2021-01-20 16:55:58
26,ISP,2021-01-26 16:56:00,0.0001,-PLSN,2021-01-26,2021-01-26 07:08:09,2021-01-26 17:03:12


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
11,ISP,2020-12-17 10:56:00,3.3000,SNPL FZFG BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47,2020-12-17 08:11:43,2020-12-17 15:26:47
12,ISP,2020-12-17 11:56:00,1.0200,+SN FZFG BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47,2020-12-17 08:11:43,2020-12-17 15:26:47
13,ISP,2020-12-17 12:56:00,0.5100,-SN FZFG BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47,2020-12-17 08:11:43,2020-12-17 15:26:47
14,ISP,2020-12-17 13:56:00,0.2500,-SN FZFG BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47,2020-12-17 08:11:43,2020-12-17 15:26:47
15,ISP,2020-12-17 14:56:00,0.0001,-SN BR BLSN,2020-12-17,2020-12-17 07:11:43,2020-12-17 16:26:47,2020-12-17 08:11:43,2020-12-17 15:26:47
23,ISP,2021-01-20 12:56:00,0.0001,-SN,2021-01-20,2021-01-20 07:12:22,2021-01-20 16:55:58,2021-01-20 08:12:22,2021-01-20 15:55:58
24,ISP,2021-01-20 13:56:00,0.5100,-SN BR,2021-01-20,2021-01-20 07:12:22,2021-01-20 16:55:58,2021-01-20 08:12:22,2021-01-20 15:55:58
31,ISP,2021-01-27 10:56:00,0.0001,-SN BR,2021-01-27,2021-01-27 07:07:21,2021-01-27 17:04:26,2021-01-27 08:07:21,2021-01-27 16:04:26
40,ISP,2021-02-01 08:56:00,0.2500,-SN BR,2021-02-01,2021-02-01 07:02:49,2021-02-01 17:10:36,2021-02-01 08:02:49,2021-02-01 16:10:36
41,ISP,2021-02-01 09:56:00,0.5100,-SN BR,2021-02-01,2021-02-01 07:02:49,2021-02-01 17:10:36,2021-02-01 08:02:49,2021-02-01 16:10:36


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.4588404761904762
2021-2022: 0.6389896551724139
2022-2023: 0.2244777777777778


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ISP,2020-12-09 19:56:00,0.0001,-SN,2020-12-09,2020-12-09 07:05:49,2020-12-09 16:25:00
1,ISP,2020-12-09 20:56:00,0.0001,-SN,2020-12-09,2020-12-09 07:05:49,2020-12-09 16:25:00
2,ISP,2020-12-16 20:56:00,0.2500,-SN,2020-12-16,2020-12-16 07:11:05,2020-12-16 16:26:26
3,ISP,2020-12-16 21:56:00,1.0200,-SN BR,2020-12-16,2020-12-16 07:11:05,2020-12-16 16:26:26
4,ISP,2020-12-16 22:56:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:11:05,2020-12-16 16:26:26
...,...,...,...,...,...,...,...
113,ISP,2021-02-19 21:56:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:41:15,2021-02-19 17:32:34
114,ISP,2021-02-19 22:56:00,0.2500,-SN BR,2021-02-19,2021-02-19 06:41:15,2021-02-19 17:32:34
115,ISP,2021-02-19 23:56:00,0.2500,-SN BR,2021-02-19,2021-02-19 06:41:15,2021-02-19 17:32:34
116,ISP,2021-02-20 00:56:00,0.5100,-SN BR,2021-02-20,2021-02-20 06:39:51,2021-02-20 17:33:45


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26932\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.6900194444444445
2021-2022: 0.15363571428571426
2022-2023: 0.361457142857143


In [33]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('SO.isp.csv', index=False)